# Tool Execution from Scratch/ No frameworks

In [ ]:
import os
import json
import requests
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr



In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)


In [ ]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    

In [ ]:
push("Hello")

In [ ]:
# Recording Connection Details (Tool 1)
#NOTE: Recording => Pushing Notification
def record_user_details(email, name="Not provided", notes="Not Provided Context"):
    push(f"User {name} with userid {email} wants to connect with context {notes}")
    return {"recorded": "ok"}

In [ ]:
# Recording unknown question (Tool 2)
def record_unknown_question(question):
    push(f"Recorded unknown question: {question}")
    return {"recorded": "ok"}

In [ ]:
# Creating json script for LLM to help it understand tool
# Most frameworks handle this automatically
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record and send Connection Requests of a person with his name, email and Notes",

    "parameters":{
        "type": "object",
        
        "properties": {
            "email": {
                "type": "string",
                "description": "Email Id of connection"
            },

            "name":{
                "type": "string",
                "description": "Name of Connection"
            },

            "notes":{
                "type": "string",
                "description": "Giving relevant context of conservation/history."
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }

}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record those questions which couldn't be answered since you didn't know the answer.",

    "parameters":{
        "type": "object",
        
        "properties": {
            "question": {
            "type": "string",
            "description": "Unknown question whose answer was not known."
            }
        },
        "required": ["question"],
        "additionalProperties": False 
    }
}

In [ ]:
tools = [{"type": "function", "function": record_user_details_json},
         {"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

In [ ]:
# This function takes list of tool calls and runs them.

def handle_tool_calls_1(tool_calls):
    results = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool used: {tool_name}", flush=True)
        
        #IF Statement!!
        if tool_name == "record_user_details":
            result = record_user_details(**arguments)

        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
        else:
            result = "Tool Error: Tool not found"        

        results.append({"role": "tool", "content":json.dumps(result), "tool_call_id": tool_call.id})

    return results        

In [ ]:
# globals(): calling a function globally with it's name.
# Input: Function calling through function name as key
# Output: dict
def handle_tool_calls_2(tool_calls):
    results = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool used: {tool_name}", flush=True)

        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {"Tool Error: Tool doesn't exist"}  #Executing if!!
        results.append({"role": "tool", "content":json.dumps(result), "tool_call_id": tool_call.id})

    return results


In [ ]:
reader = PdfReader("Profile.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    # print(text)
    if text:
        linkedin += text

with open("summary.txt", "r", encoding = "utf-8") as f:
    summary = f.read()

name = "Dhruv Tiwari"    

In [ ]:
system_prompt = f"""You are acting as {name}. You are answering questions related to {name}'s career, background, skills \n
and experience on {name}'s website. Your task is to represent yourself as {name} and try to be as faithful as \n
possible. You are given a summary of {name}'s background and LinkedIn profile which you can use to answer. \n
Respond as if you are responding to a future client who came across website.Do not reply with actions.\n
If you don't know an answer to a question, use your tool record_unknown_question for recording that querry.\n
If user is engaging in discussion, tell him to get in touch via email and record his email, name and some past conversations and record these details\n
through your tool record_user_details. 
"""
system_prompt += f"\n\n ##Summary: \n{summary} \n\n ##LinkedIn Profile: \n {linkedin}"
system_prompt += f"\n\n Based on this context, answer the querries as {name}"

In [ ]:
gemini = OpenAI(api_key= os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    
    while not done:
        response = gemini.chat.completions.create(model = "gemini-2.5-flash", messages=messages, tools=tools)
        if response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            tool_result = handle_tool_calls_2(msg.tool_calls)
            print(tool_result)
            messages.append(msg)
            messages.extend(tool_result)
        else:
            done = True
    
    return response.choices[0].message.content    
    

In [ ]:
gr.ChatInterface(chat, title="Dhruv's Professional Chatbot").launch()